# Heat Equation

The fundamental goal of model reduction is to efficiently make physics-based predictions. Given synthetic or experimental data that was generated or collected under a certain set of conditions, we aim to construct a cost-effective model that produces accurate solutions under new sets of conditions. This tutorial explores the following prediction problems for the heat equation example of {cite}`PW2016OperatorInference`:
1. Predicting **forward in time**.
2. Using new time-dependent **boundary conditions**.
3. Changing the **system parameters** (e.g., coefficients in the governing equation).

## Problem Statement

:::{admonition} Governing Equations
:class: attention

Let $\Omega = [0,L]\subset \mathbb{R}$ be the spatial domain indicated by the variable $x$, and let $[0,T]\subset\mathbb{R}$ be the time domain with variable $t$. We consider the one-dimensional heat equation with non-homogeneous Dirichlet boundary conditions,

\begin{align*}
    &\frac{\partial}{\partial t} q(x,t;\mu) = \mu\frac{\partial^2}{\partial x^2}q(x,t;\mu)
    & x &\in\Omega,\quad t\in[0,T],
    \\
    &q(0,t;\mu) = q(L,t;\mu) = u(t)
    & t &\in[0,T],
    \\
    &q(x,0;\mu) = \big(e^{\alpha(x - 1)} + e^{-\alpha x} - e^{-\alpha}\big)u(0)
    & x &\in \Omega,
\end{align*}

where the constant $\mu > 0$ is the thermal diffusivity and where $q(x,t;\mu)$ is the unknown state variable. This is a model for a one-dimensional rod conducting heat with a fixed initial heat profile. The temperature at the ends of the rod are governed by the input function $u(t)$, but  heat is allowed to diffuse through the rod and flow out at the ends of the domain. We aim to numerically solve for $q(x,t;\mu)$ efficiently for all $t \in [0,T]$ and/or for various choices of $u(t)$ and $\mu$.
:::

:::{note}
This problem can be solved with a straightforward discretization of the spatial domain $\Omega$ without little computational effort, so using model reduction to speed up the computation is not highly beneficial. However, the way that the user interacts with the package for this problem is highly similar for more complex problems.
:::

## Prediction in Time

Our first objective is to get solutions in time beyond a set of available training data.

:::{image} ../../images/summary.svg
:align: center
:width: 80 %
:::

:::{admonition} Objective
:class: attention

Construct a reduced-order model (ROM) of the heat equation that is **predictive in time**. In other words, we will observe data for $t \in [0, T']$ with $T' < T$, use that data to construct the ROM, and use the ROM to predict the solution for the entire time domain $[0,T]$.
:::

### Full-order Model Definition

As in the last tutorial, we use a centered finite difference approximation for the spatial derivative to arrive at a first-order system, this time of the form

$$
    \frac{\text{d}}{\text{d}t}\mathbf{q}(t;\mu)
    = \mathbf{A}(\mu)\mathbf{q}(t;\mu) + \mathbf{B}(\mu)u(t),
    \qquad
    \mathbf{q}(0;\mu)
    = \mathbf{q}_0.
$$ (eq_heat_fom_parametric)

:::{dropdown} Discretization details

We take an equidistant grid $\{x_i\}_{i=0}^{n+1} \subset \Omega$,

\begin{align*}
    0 &= x_0 < x_1 < \cdots < x_n < x_{n+1} = L
    &
    &\text{and}
    &
    \delta x &= \frac{L}{n+1} = x_{i+1} - x_{i},\quad i=1,\ldots,n-1.
\end{align*}

The boundary conditions prescribe $q(x_0,t) = q(x_{n+1},t) = u(t)$. Our goal is to compute $q(x,t)$ at the interior spatial points $x_{1},x_{2},\ldots,x_{n}$ for various $t\in[0,T]$, so we consider the state vector $\mathbf{q}(t) = [~q(x_{1}, t)~\cdots~q(x_{n}, t)~]^{\top}\in\mathbb{R}^n$ and derive a system governing the evolution of $\mathbf{q}(t)$ in time.

Approximating the spatial derivative with a central finite difference approximation,

$$
    \frac{\partial^2}{\partial x^2}q(x,t)
    \approx \frac{q(x-\delta x,t) - 2q(x,t) + q(x+\delta x,t)}{(\delta x)^2},
$$

we arrive at the following matrices for the full-order model.

\begin{align*}
    \mathbf{A}(\mu) &= \frac{\mu}{(\delta x)^2}\left[\begin{array}{ccccc}
        -2 & 1 & & & \\
        1 & -2 & 1 & & \\
        & \ddots & \ddots & \ddots & \\
        & & 1 & -2 & 1 \\
        & & & 1 & -2 \\
    \end{array}\right] \in\mathbb{R}^{n\times n},
    &
    \mathbf{B}(\mu) &= \frac{\mu}{(\delta x)^2}\left[\begin{array}{c}
        1 \\ 0 \\ \vdots \\ 0 \\ 1
    \end{array}\right]\in\mathbb{R}^{n}.
\end{align*}
:::

The state $\mathbf{q}(t;\mu)$ implicity depends on the parameter $\mu$ because the operators $\mathbf{A}(\mu)$ and $\mathbf{B}(\mu)$ are parameterized by $\mu$.
For now, we set $\mu = 1$ and simply write

$$
    \frac{\text{d}}{\text{d}t}\mathbf{q}(t)
    = \mathbf{A}\mathbf{q}(t) + \mathbf{B}u(t),
    \qquad
    \mathbf{q}(0)
    = \mathbf{q}_0.
$$

This is the _full-order model_ (FOM), which we will use to generate training data for the time domain $[0, T'] \subset [0, T]$.

### Training Data Generation

Let $L = T = \mu = 1$, $\alpha = 100$, and suppose for now that the boundary conditions are given by the constant input function $u(t) \equiv 1$.
We begin by simulating the full-order system described above with a uniform time step $\delta t = 10^{-3}$, yielding $10^3 + 1 = 1001$ total time steps (1000 steps past the initial condition).
We will assume that we can only observe the first $k = 100$ time steps and use the ROM to predict the remaining $901$ steps.

In [ ]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt

import rom_operator_inference as opinf

In [ ]:
# Matplotlib customizations.
plt.rc("axes.spines", right=False, top=False)
plt.rc("figure", dpi=300, figsize=(9, 3))
plt.rc("font", family="serif")
plt.rc("legend", edgecolor="none", frameon=True)
plt.rc("text", usetex=True)

# Pandas display options.
pd.options.display.float_format = "{:.4%}".format

In [ ]:
# Construct the spatial domain.
L = 1                           # Spatial domain length.
n = 2**7 - 1                    # Spatial grid size.
x_all = np.linspace(0, L, n+2)  # Full spatial grid.
x = x_all[1:-1]                 # Interior spatial grid (where q is unknown).
dx = x[1] - x[0]                # Spatial resolution.

# Construct the temporal domain.
T = 1                           # Temporal domain length (final simulation time).
K = T*10**3 + 1                 # Temporal grid size.
t = np.linspace(0, T, K)        # Temporal grid.
dt = t[1] - t[0]                # Temporal resolution.

print(f"Spatial step size δx = {dx}")
print(f"Temporal step size δt = {dt}")

In [ ]:
# Construct the full-order state matrix A.
dx2inv = 1 / dx**2
diags = np.array([1, -2, 1]) * dx2inv
A = sparse.diags(diags, [-1, 0, 1], (n, n))

# Construct the full-order input matrix B.
B = np.zeros_like(x)
B[0], B[-1] = dx2inv, dx2inv

In [ ]:
# Define the inputs.
input_func = np.ones_like       # Constant input function u(t) = 1.
U_all = input_func(t)           # Inputs over the time domain.

# Construct the initial condition.
alpha = 100
q0 = np.exp(alpha*(x-1)) + np.exp(-alpha*x) - np.exp(-alpha)

print(f"shape of A:\t{A.shape}")
print(f"shape of B:\t{B.shape}")
print(f"shape of q0:\t{q0.shape}")

Since this is a diffusive problem, we will use the implicit (backward) Euler method for solving the ODEs.
For the problem $\frac{\text{d}}{\text{d}t}\mathbf{q}(t) = \mathbf{F}(t, \mathbf{q}(t), \mathbf{u}(t))$, implicit Euler is defined by the rule

$$
    \mathbf{q}_{j+1} = \mathbf{q}_{j} + \delta t \mathbf{F}(t_{j+1},\mathbf{q}_{j+1},u_{j+1}),
$$

where $\mathbf{q}_{j} := \mathbf{q}(t_{j})$ and $u_{j} := u(t_{j})$.
With the form $\mathbf{F}(t,\mathbf{q}(t),u(t)) = \mathbf{A}\mathbf{q}(t) + \mathbf{B}u(t)$, this becomes

$$
    \mathbf{q}_{j+1} = (\mathbf{I} - \delta t \mathbf{A})^{-1}\left(\mathbf{q}_{j} + \delta t \mathbf{B} u_{j+1}\right),
$$

where $\mathbf{I}$ is the identity matrix.

In [ ]:
def implicit_euler(t, q0, A, B, U):
    """Solve the system

        dq / dt = Aq(t) + Bu(t),    q(0) = q0,

    over a uniform time domain via the implicit Euler method.

    Parameters
    ----------
    t : (k,) ndarray
        Uniform time array over which to solve the ODE.
    q0 : (n,) ndarray
        Initial condition.
    A : (n, n) ndarray
        State matrix.
    B : (n,) or (n, 1) ndarray
        Input matrix.
    U : (k,) ndarray
        Inputs over the time array.

    Returns
    -------
    q : (n, k) ndarray
        Solution to the ODE at time t; that is, q[:,j] is the
        computed solution corresponding to time t[j].
    """
    # Check and store dimensions.
    k = len(t)
    n = len(q0)
    B = np.ravel(B)
    assert A.shape == (n, n)
    assert B.shape == (n,)
    assert U.shape == (k,)
    I = np.eye(n)

    # Check that the time step is uniform.
    dt = t[1] - t[0]
    assert np.allclose(np.diff(t), dt)

    # Factor I - dt*A for quick solving at each time step.
    factored = la.lu_factor(I - dt*A)

    # Solve the problem at each time step.
    q = np.empty((n, k))
    q[:,0] = q0.copy()
    for j in range(1, k):
        q[:, j] = la.lu_solve(factored, q[:, j-1] + dt*B*U[j])

    return q

In [ ]:
# Compute snapshots by solving the equation with implicit_euler().
Q_all = implicit_euler(t, q0, A, B, U_all)

# Retain only the first k snapshots/inputs for training the ROM.
k = 100                         # Number of training snapshots.
t_train = t[:k]                 # Temporal domain for training snapshots.
Q = Q_all[:, :k]                # Training snapshots.
U = U_all[:k]                   # Inputs corresponding to the training snapshots.

# Also compute time derivatives (dq/dt) at each training snapshot.
Qdot = A @ Q + B.reshape((-1,1))*U

print(f"shape of Q:\t{Q.shape}")
print(f"shape of Qdot:\t{Qdot.shape}")

Finally, we visualize the snapshots to get a sense of how the solution looks qualitatively.

In [ ]:
def plot_heat_data(Z, title, ax=None):
    """Visualize temperature data in space and time."""
    if ax is None:
        _, ax = plt.subplots(1, 1)

    # Plot a few snapshots over the spatial domain.
    sample_columns = [0, 10, 20, 40, 80, 160, 320, 640]
    sample_columns = [0] + [2**d for d in range(10)]
    color = iter(plt.cm.viridis_r(np.linspace(.05, 1, len(sample_columns))))
    while sample_columns[-1] > Z.shape[1]:
        sample_columns.pop()
    leftBC, rightBC = [input_func(x_all[0])], [input_func(x_all[-1])]
    for j in sample_columns:
        q_all = np.concatenate([leftBC, Z[:,j], rightBC])
        ax.plot(x_all, q_all, color=next(color), label=fr"$q(x,t_{{{j}}})$")

    ax.set_xlim(x_all[0], x_all[-1])
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$q(x,t)$")
    ax.legend(loc=(1.05, .05))
    ax.set_title(title)

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2)
plot_heat_data(Q, "Snapshot data for training", ax1)
plot_heat_data(Q_all, "Full-order model solution", ax2)
ax1.legend([])
plt.show()

### ROM Construction

Now that we have snapshot data $\mathbf{Q} \in \mathbb{R}^{n \times k}$, we can construct [a basis](sec-basis-computation) $\mathbf{V}_r \in \mathbb{R}^{n \times r}$ to use in the construction of the ROM. For operator inference (OpInf), we often use the [proper orthogonal decomposition](subsec-pod) (POD) basis. The integer $r$, which defines the dimension of the reduced-order model to be constructed, is usually determined by how quickly the singular values of $\mathbf{Q}$ decay. In this example, we choose the minimal $r$ such that the [residual energy](subsec-basis-size) is less than a given tolerance $\varepsilon$, i.e.,

$$
\frac{\sum_{j=r + 1}^{k}\sigma_{j}^{2}}{\sum_{j=1}^{k}\sigma_{j}^{2}} = \frac{||\mathbf{Q} - \mathbf{V}_r \mathbf{V}_r^{\top}\mathbf{Q}||_{F}^{2}}{||\mathbf{Q}||_{F}^{2}} < \varepsilon.
$$

In [ ]:
# Compute the largest possible basis and all singular values.
V, svdvals = opinf.pre.pod_basis(Q)

# Check the decay of the residual energy based on the singular values.
r = opinf.pre.residual_energy(svdvals, tol=1e-10, plot=True)
plt.xlim(0, 25)

Vr = V[:, :r]
print(f"r = {r:d} singular values required so residual energy < 1e-10")

::::{margin}
:::{note}
In this case, since $u(t) \equiv 1$ is constant, we could equivalently set `modelform="cA"` to learn a ROM of the form $\frac{\text{d}}{\text{d}t}\widehat{\mathbf{q}}(t) = \widehat{\mathbf{c}} + \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t)$, where $\widehat{\mathbf{c}}$ is a constant term.
There is no difference between the two models, i.e., $\widehat{\mathbf{c}} = \widehat{\mathbf{B}}u(t) = \widehat{\mathbf{B}}$, except that `modelform="AB"` allows us to use different inputs for $u(t)$ later on.
:::
::::

Now we can learn the reduced model with OpInf.
Because the full-order model is of the form $\frac{\text{d}}{\text{d}t}\mathbf{q}(t) = \mathbf{A}\mathbf{q}(t) + \mathbf{B}u(t)$, we set the form of the ROM to $\frac{\text{d}}{\text{d}t}\widehat{\mathbf{q}}(t) = \widehat{\mathbf{A}}\widehat{\mathbf{q}}(t) + \widehat{\mathbf{B}}u(t)$ by specifying `modelform="AB"`.

In [ ]:
# Train the model.
rom = opinf.ContinuousOpInfROM(modelform="AB")
rom.fit(basis=Vr, states=Q, ddts=Qdot, inputs=U)

### ROM Evaluation

Like the FOM, we integrate the learned ROM using the implicit Euler method, using the reduced-order operators $\widehat{\mathbf{A}}$ and $\widehat{\mathbf{B}}$ and the projected initial condition $\widehat{\mathbf{q}}_{0} = \mathbf{V}^{\mathsf{T}}\mathbf{q}_{0}$.
The resulting low-dimensional state vectors are translated back to the full-dimensional space via $\mathbf{q}(t) = \mathbf{V}_{r}\widehat{\mathbf{q}}(t)$.

In [ ]:
q0_ = Vr.T @ q0                                     # Project the initial condition.
Q_ROM = Vr @ implicit_euler(t, q0_, rom.A_.entries, rom.B_.entries, U_all)

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2)
plot_heat_data(Q_ROM, "Reduced-order model solution", ax1)
plot_heat_data(Q_all, "Full-order model solution", ax2)
ax1.legend([])
plt.show()

To quantify the accuracy of the ROM, we evaluate the ROM solution error in the Frobenius norm and compare it to the projection error,

$$
    \text{err}_{\text{ROM}}
    = \frac{||\mathbf{Q}_{\text{all}} - \mathbf{Q}_{\text{ROM}}||_F}{||\mathbf{Q}_{\text{all}}||_F},
    \qquad
    \text{err}_{\text{proj}}
    = \frac{||\mathbf{Q}_{\text{all}} - \mathbf{V}_{r}\mathbf{V}_{r}^{\top}\mathbf{Q}_{\text{all}}||_F}{||\mathbf{Q}_{\text{all}}||_F},
$$

where $\mathbf{Q}_{\text{all}}$ is the full-order model solution over the entire time domain and $\mathbf{Q}_{\text{ROM}}$ is the reduced-order model solution.

In [ ]:
rel_froerr_projection = opinf.post.projection_error(Q_all, rom.basis)[1]
rel_froerr_opinf = opinf.post.frobenius_error(Q_all, Q_ROM)[1]

print("Relative Frobenius-norm errors", '-'*33,
      f"projection error:\t{rel_froerr_projection:%}",
      f"OpInf ROM error:\t{rel_froerr_opinf:%}",
      sep='\n')

The ROM error cannot be better than the projection error, but it is pretty close to it. We also compare the ROM error with the projection error as a function of time, i.e.,

$$
    \text{err}_{\text{ROM}}(t)
    = \frac{\|\mathbf{q}(t) - \mathbf{q}_{\text{ROM}}(t)\|_{2}}{\|\mathbf{q}(t)\|_{2}},
    \qquad
    \text{err}_{\text{proj}}(t)
    = \frac{\|\mathbf{q}(t) - \mathbf{V}_{r}\mathbf{V}_{r}^{\mathsf{T}}\mathbf{q}(t)\|_{2}}{\|\mathbf{q}(t)\|_{2}},
$$

where $\mathbf{q}(t)$ is the full-order solution and $\mathbf{q}_{\text{ROM}}(t)$ is the ROM solution at time $t$.

:::{tip}
In this problem, $\mathbf{q}(t) \to \mathbf{0}$ as $t$ increases, so a relative error may not be appropriate since $\|\mathbf{q}(t)\|_{2}$ appears in the denominator.
In situations like this, consider using the _normalized absolute error_ by replacing the denominator with $\max_{\tau\in[0,T]}\|\mathbf{q}(t)\|$, for example:
$$
    \text{err}_{\text{ROM}}(t)
    = \frac{\|\mathbf{q}(t) - \mathbf{q}_{\text{ROM}}(t)\|_{2}}{\max_{\tau\in[0,T]}\|\mathbf{q}(\tau)\|_{2}}.
$$

Use `normalize=True` in `opinf.post.lp_error()` to use this error measure instead of the relative error.
:::

In [ ]:
projerr_in_time = opinf.post.lp_error(Q_all,
                                      Vr @ (Vr.T @ Q_all),
                                      normalize=True)[1]

def plot_errors_over_time(Zlist, labels):
    """Plot normalized absolute projection error and ROM errors
    as a function of time.

    Parameters
    ----------
    Zlist : list((n, k) ndarrays)
        List of reduced-order model solutions.
    labels : list(str)
        Labels for each of the reduced-order models.
    """
    fig, ax = plt.subplots(1, 1)
    
    ax.semilogy(t, projerr_in_time, "C3", label="Projection Error")
    colors = ["C0", "C5"]
    for Z, label, c in zip(Zlist, labels, colors[:len(Zlist)]):
        rel_err = opinf.post.lp_error(Q_all, Z, normalize=True)[1]
        plt.semilogy(t, rel_err, c, label=label)

    ax.set_xlabel(r"$t$")
    ax.set_ylabel("Normalized absolute error")
    ax.legend(loc="lower right")
    plt.show()

In [ ]:
plot_errors_over_time([Q_ROM], ["ROM Error"])

### Comparison with Intrusive Projection

Under some idealized assumptions, as the amount of training data $k$ and the dimension $r$ increases, the reduced operators $\widehat{\mathbf{A}}$ and $\widehat{\mathbf{B}}$ learned through OpInf converge to the corresponding operators obtained through _intrusive projection_,

\begin{align*}
    \widetilde{\mathbf{A}} &= \mathbf{V}_{r}^{\mathsf{T}} \mathbf{A} \mathbf{V}_{r},
    &
    \widetilde{\mathbf{B}} &= \mathbf{V}_{r}^{\mathsf{T}}\mathbf{B}.
\end{align*}

The operation is called "intrusive" because it requires explicit access to the full-order operators $\mathbf{A}$ and $\mathbf{B}$.

In [ ]:
Atilde = Vr.T @ A @ Vr
Btilde = Vr.T @ B
Q_ROM_intrusive = Vr @ implicit_euler(t, q0_, Atilde, Btilde, U_all)

:::{tip}
The OpInf ROM classes, such as `ContinuousOpInfROM`, learn reduced-order model operators intrusively whenever the full-order operators are provided through the `known_operators` argument of `fit()`.
This argument should be a dictionary mapping the modelform key to the full-order operator.
If every full-order operator is provided, then the `states` and `ddts` arguments of `fit()` may be set to `None`, since there is no data-driven learning in this case.

```python
>>> rom_intrusive = opinf.ContinuousOpInfROM("AB")
>>> rom_intrusive.fit(Vr, None, None, known_operators={"A": A, "B": B})
>>> np.all(rom_intrusive.A_.entries == Atilde)
False
>>> np.all(rom_intrusive.B_.entries == Btilde)
True

>>> Q_ROM_intrusive = Vr @ implicit_euler(t, q0_, rom_intrusive.A_.entries, rom_intrusive.B_.entries, U_all)
```

If, for example, only the full-order operator $\mathbf{B}$ were known (but not $\mathbf{A}$), we can learn $\widehat{\mathbf{A}}$ through OpInf and $\widehat{\mathbf{B}}$ through intrusive projection:

```python
>>> rom_partially_intrusive = opinf.ContinuousOpInfROM("AB")
>>> rom_partially_intrusive.fit(Vr, Q, Qdot, known_operators={"B": B})
>>> np.all(rom_partially_intrusive.A_.entries == Atilde)
False
>>> np.all(rom_partially_intrusive.B_.entries == Btilde)
True
```

:::

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2)
plot_heat_data(Q_ROM, "OpInf ROM solution", ax1)
plot_heat_data(Q_ROM_intrusive, "Intrusive ROM solution", ax2)
ax1.legend([])
plt.show()

In [ ]:
rel_froerr_intrusive = opinf.post.frobenius_error(Q_all, Q_ROM_intrusive)[1]

print("Relative Frobenius-norm errors", '-'*33,
      f"projection error:\t{rel_froerr_projection:%}",
      f"OpInf ROM error:\t{rel_froerr_opinf:%}",
      f"intrusive ROM error:\t{rel_froerr_intrusive:%}",
      sep='\n')

In [ ]:
plot_errors_over_time([Q_ROM, Q_ROM_intrusive],
                      ["OpInf ROM Error", "Intrusive ROM Error"])

Let's repeat the experiment with different choices of $r$ to see how the size of the ROM affects its accuracy.

In [ ]:
V, _ = opinf.pre.pod_basis(Q)

def run_trial(r):
    """Do OpInf / intrusive ROM prediction with r basis vectors."""
    Vr = V[:,:r]
    q0_ = Vr.T @ q0

    # Construct and simulate the intrusive ROM.
    rom_intrusive = opinf.ContinuousOpInfROM("AB").fit(Vr, None, None,
                                                       known_operators={"A":A, "B":B})
    Q_ROM_intrusive = Vr @ implicit_euler(t, q0_,
                                          rom_intrusive.A_.entries,
                                          rom_intrusive.B_.entries, U_all)

    # Construct and simulate the inferred ROM.
    rom_opinf = opinf.ContinuousOpInfROM("AB").fit(Vr, Q, Qdot, U)
    Q_ROM_opinf = Vr @ implicit_euler(t, q0_,
                                      rom_opinf.A_.entries,
                                      rom_opinf.B_.entries, U_all)

    # Calculate errors.
    projection_error = opinf.pre.projection_error(Q_all, Vr)[1]
    intrusive_error = opinf.post.frobenius_error(Q_all, Q_ROM_intrusive)[1]
    opinf_error = opinf.post.frobenius_error(Q_all, Q_ROM_opinf)[1]

    return projection_error, intrusive_error, opinf_error

In [ ]:
def plot_state_error(rmax, runner, ylabel):
    """Run the experiment for r = 1, ..., rmax and plot results."""
    rs = np.arange(1, rmax+1)
    err_projection, err_intrusive, err_opinf = zip(*[runner(r) for r in rs])

    fig, ax = plt.subplots(1, 1)
    ax.semilogy(rs, err_projection, 'C3-',
                label="projection error", lw=1)
    ax.semilogy(rs, err_opinf, 'C0o-',
                label="OpInf ROM error", lw=1, mfc="none", mec="C0", mew=1.5)
    ax.semilogy(rs, err_intrusive, 'C5+-',
                label="intrusive ROM error", lw=1, mew=2)

    ax.set_xlim(rs.min(), rs.max())
    ax.set_xticks(rs, [str(int(r)) for r in rs])
    ax.set_xlabel(r"Reduced dimension $r$")
    ax.set_ylabel(ylabel)
    ax.legend(loc="upper right", fontsize=14, framealpha=1)
    ax.grid(ls=':')
    plt.show()

In [ ]:
plot_state_error(20, run_trial, "Relative Frobenius-norm error")

:::{admonition} Takeaway
:class: attention
In this case, the inferred and projected ROMs give essentially the same result.
However, the inferred ROM successfully emulates the FOM **without explicit knowledge of the operators** $\mathbf{A}$ **and** $\mathbf{B}$.
:::

## New Boundary Conditions

New BC functions to try:

\begin{align*}
    u(t) &= 1 - \frac{1}{2}\sin(\pi t)
    &
    u(t) &= 1 - \frac{1}{2}\sin(3\pi t)
    \\
    u(t) &= 1 - \frac{1}{2}\sin(7 \pi t)
    &
    u(t) &= 1 + \frac{1}{2}t(t - 1)
    \\
    u(t) &= 1 + 25 (t (t - 1))^{3}
    &
    u(t) &= 1 + 500 (t (t - 1))^{5}
    \\
    u(t) &= 1 + 1000 (t (t - 1))^{7}
    &
    u(t) &= 1 + 10000 (t (t - 1))^{9}
\end{align*}

## Prediction in Parameter Space

Recall the governing equation,

$$
    \frac{\partial}{\partial t} q(x,t;{\color{teal}\mu})
    = {\color{teal}\mu}\frac{\partial^2}{\partial x^2}q(x,t;{\color{teal}\mu}).
$$

In this section we examine the role of the constant $\mu > 0$, the heat diffusivity parameter.

:::{admonition} Objective
:class: attention

Construct a ROM of the heat equation that can be solved for different choices of the diffusivity parameter $\mu > 0$.
We will observe data for a few values of $\mu$ and use the ROM to predict the solution for new values of $\mu$. As before, we also aim to be predictive in time.
:::

### Full-order Model Definition

We solved this problem earlier for fixed $\mu = 1$.
For variable $\mu$, {eq}`eq_heat_fom_parametric` defines the full-order model:

$$
    \frac{\text{d}}{\text{d}t}\mathbf{q}(t;\mu)
    = \mathbf{A}(\mu)\mathbf{q}(t;\mu) + \mathbf{B}(\mu)u(t),
    \qquad
    \mathbf{q}(0;\mu)
    = \mathbf{q}_0.
$$

Note that $\mathbf{A}(\mu) = \mu \mathbf{A}(1)$ and $\mathbf{B}(\mu) = \mu \mathbf{B}(1)$, and that $\mathbf{A}(1)$ and $\mathbf{B}(1)$ are the full-order operators we constructed previously.

### Training Data Generation

We consider the parameter domain $\mathcal{D} = [.1, 10] \subset \mathbb{R}$.
Taking $s$ logarithmically spaced samples $\{\mu_i\}_{i=1}^{s}\subset\mathcal{D}$, we solve the full-order model over $[0, T']$ for each parameter sample.
For each parameter $\mu_{i}$, the resulting snapshots matrix is denoted as $\mathbf{Q}(\mu_{i})\in \mathbb{R}^{n \times k}$.
We choose $s = 10$ training parameters in the following experiment.

In [ ]:
s = 10                          # Number of parameter samples.
params = np.logspace(-1, 1, s)  # Get s logarithmically spaced values of µ from D = [.1, 10].

# Retain only the first k snapshots/inputs for training the ROM.
k = 500                         # Number of training snapshots.
t_train = t[:k]                 # Temporal domain for training snapshots.
U = U_all[:k]                   # Inputs corresponding to the training snapshots.

# Solve the full-order model at each of the parameter samples.
Qs, Qdots, Us = [], [], []
for µ in params:
    Aµ, Bµ = µ * A, µ * B
    Qµ = implicit_euler(t_train, q0, Aµ, Bµ, U)  # implicit_euler(t_train, q0, Aµ, Bµ, U_all)
    Qdotµ = Aµ @ Qµ + Bµ.reshape((-1,1)) * U  # U_all
    Qs.append(Qµ)
    Qdots.append(Qdotµ)
    Us.append(U)  # U_all)

### ROM Construction

A (global) POD basis can be constructed from the concatenation of the individual snapshot matrices,

$$
    \mathbf{Q}
    = \left[~\mathbf{Q}(\mu_1)~\cdots~\mathbf{Q}(\mu_s)~\right]
    \in\mathbb{R}^{n \times sk}.
$$

We can select the reduced dimension $r$ as before by examining the residual energy of the singular values.

In [ ]:
V, svdvals = opinf.pre.pod_basis(np.hstack(Qs))
opinf.pre.residual_energy(svdvals, tol=1e-10, plot=True)
plt.xlim(right=40)
plt.show()

Alternatively, we could choose $r$ so that the average relative projection error,

$$
    \text{avgerr}_\text{proj} = \frac{1}{s}\sum_{i=1}^{s}\frac{||\mathbf{Q}(\mu_i) - \mathbf{V}_r \mathbf{V}_r^{\top}\mathbf{Q}(\mu_i)||_F}{||\mathbf{Q}(\mu_i)||_F},
$$

is below a certain threshold, say $10^{-5}$.

In [ ]:
def average_relative_projection_error(r):
    """Compute the average relative projection error with r basis vectors."""
    Vr = V[:, :r]
    return np.mean([opinf.pre.projection_error(Q, Vr)[1] for Q in Qs])

In [ ]:
fig, ax = plt.subplots(1, 1)
rs = np.arange(1, 21)
ax.axhline(1e-5, color="gray", lw=1)
ax.axvline(11, color="gray", lw=1)
ax.semilogy(rs, [average_relative_projection_error(r) for r in rs],
            "C3.-", ms=10)
ax.set_xticks(rs[::2])
ax.set_xlabel(r"$r$")
ax.set_ylabel("Average relative\nprojection error")
plt.show()

Based on these criteria, we choose $r = 11$.

In [ ]:
r = 9
Vr = V[:, :r]

### Interpolatory Operator Inference

There are several strategies to account for the parameter $\mu$. The reduced-order operators obtained through projection are given by

$$
    \widetilde{\mathbf{A}}(\mu)
    = \mathbf{V}_{r}^{\mathsf{T}} \mathbf{A}(\mu) \mathbf{V}_{r},
    \qquad
    \widetilde{\mathbf{B}}(\mu)
    = \mathbf{V}_{r}^{\mathsf{T}} \mathbf{B}(\mu).
$$

Here, we perform interpolation on the entries of the reduced-order operators learned for each parameter sample. This means we learn a separate ROM for each $\mu_i$, $i=1, \ldots, s$, obtaining reduced-order operators $\widehat{\mathbf{A}}(\mu_{i})$ and $\widehat{\mathbf{B}}(\mu_{i})$.
Then, for a new parameter value $\bar{\mu}\in\mathcal{D}$, we interpolate the entries of the learned reduced model operators to create a new reduced model corresponding to $\bar{\mu}\in\mathcal{D}$.
<!-- 
$$
    \widehat{\mathbf{A}}(\bar{\mu})_{i,j}
    = \text{interpolate}\left(\mathbf{A}(\mu_{1})_{i,j}, \ldots, \mathbf{A}(\mu_{s})_{i,j}; \bar{\mu}\right)
$$
 -->
The `InterpolatedContinuousOpInfROM` class encapsulates this process.

In [ ]:
# Learn reduced models for each parameter value.
rom = opinf.InterpolatedContinuousOpInfROM("AB")
rom.fit(basis=Vr, parameters=params, states=Qs, ddts=Qdots, inputs=Us)

:::{tip}
**TODO**: we can learn an interpolatory intrusive model this way.

```python
>>> rom_intrusive = opinf.InterpolatedContinuousOpInfROM("AB")
>>>rom_intrusive.fit(Vr, params, None, None,
                      known_operators={"A": [µ * A for µ in params],
                                       "B": [µ * B for µ in params]})
```
:::

### ROM Evaluation

To test the ROM, we take $s$ logarithmically distributed random samples in $\mathcal{D}$ and compute the average relative state error,

$$
    \text{avgerr}_\text{ROM} = \frac{1}{s}\sum_{i=1}^{s}\frac{||\mathbf{Q}(\mu_i) - \mathbf{Q}_{\text{ROM}}(\mu_i)||_F}{||\mathbf{Q}(\mu_i)||_F},
$$

where $\mathbf{Q}_{\text{ROM}}(\mu_{i})$ is the ROM solution at parameter $\mu_{i}$.

In [ ]:
params_test = np.sort(10**np.random.uniform(-1, 1, s))
params_test

In [ ]:
def run_trial_parametric(r):
    """Do OpInf / intrusive ROM prediction with r basis vectors."""
    Vr = V[:,:r]
    q0_ = Vr.T @ q0

    # Learn an OpInf ROMs from the training data.
    rom_opinf = opinf.InterpolatedContinuousOpInfROM("AB")
    rom_opinf.fit(basis=Vr, parameters=params, states=Qs, ddts=Qdots, inputs=Us)

    # Test the 
    projection_error, intrusive_error, opinf_error = 0, 0, 0
    for µ in params_test:

        # Solve the FOM at this parameter value.
        Aµ = µ * A
        Bµ = µ * B
        Q_FOM = implicit_euler(t, q0, Aµ, Bµ, U_all)

        # Simulate the intrusive ROM at this parameter value.
        Q_ROM_intrusive = Vr @ implicit_euler(t, q0_, Vr.T @ Aµ @ Vr, Vr.T @ Bµ, U_all)

        # Simulate the interpolating OpInf ROM at this parameter value.
        romµ = rom_opinf(µ)
        Q_ROM_opinf = Vr @ implicit_euler(t, q0_, romµ.A_.entries, romµ.B_.entries, U_all)

        # Calculate errors.
        projection_error += opinf.pre.projection_error(Q_FOM, Vr)[1]
        intrusive_error += opinf.post.frobenius_error(Q_FOM, Q_ROM_intrusive)[1]
        opinf_error += opinf.post.frobenius_error(Q_FOM, Q_ROM_opinf)[1]

    # Average the relative errors.
    projection_error /= len(params_test)
    intrusive_error /= len(params_test)
    opinf_error /= len(params_test)

    return projection_error, intrusive_error, opinf_error

In [ ]:
plot_state_error(20, run_trial_parametric, "Average relative\nFrobenius-norm error")